In [14]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

# 1️⃣ Load dataset (only top 10,000 most frequent words)
max_words = 10_000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_words)

# 2️⃣ Pad sequences so all reviews are same length
max_len = 200
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test  = sequence.pad_sequences(x_test,  maxlen=max_len)

# 3️⃣ Build RNN model
model = Sequential([
    layers.Embedding(input_dim=max_words, output_dim=64, input_length=max_len),
    layers.LSTM(128, activation="tanh"),
    layers.Dense(1, activation="sigmoid")
])

# 4️⃣ Compile model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# 5️⃣ Train the model
history = model.fit(
    x_train, y_train,
    epochs=5,
    batch_size=64,
    validation_split=0.2,
    verbose=1
)

# 6️⃣ Evaluate on test data
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Test accuracy: {acc:.3f}")


Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 98s 299ms/step - accuracy: 0.7372 - loss: 0.5093 - val_accuracy: 0.8520 - val_loss: 0.3568
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 91s 290ms/step - accuracy: 0.8783 - loss: 0.3002 - val_accuracy: 0.8516 - val_loss: 0.3430
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 94s 299ms/step - accuracy: 0.9194 - loss: 0.2085 - val_accuracy: 0.8496 - val_loss: 0.3676
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 92s 295ms/step - accuracy: 0.9432 - loss: 0.1593 - val_accuracy: 0.8734 - val_loss: 0.3809
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 94s 301ms/step - accuracy: 0.9575 - loss: 0.1191 - val_accuracy: 0.8628 - val_loss: 0.3776
Test accuracy: 0.858


In [20]:
# 🧰 Step 1: Import necessary libraries
import re, numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

# ⚙️ Step 2: Define important constants (same as used during training)
max_words = 10_000   # use top 10,000 common words
max_len   = 200      # every review = 200 words
index_from = 3       # IMDB reserves 0,1,2 for special tokens

# 🗺️ Step 3: Load IMDB's word index and adjust it (+3 shift)
raw_word_index = imdb.get_word_index()
word_index = {w:(i+index_from) for w,i in raw_word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

# ✂️ Step 4: Make a simple word splitter (keeps only letters and apostrophes)
_tok = re.compile(r"[a-zA-Z']+")

# 🔢 Step 5: Convert a text review into a sequence of numbers
def encode_review(text, max_words=max_words, max_len=max_len):
    tokens = _tok.findall(text.lower())   # make lowercase & split into words
    seq = [1]  # start with <START> token
    for t in tokens:
        idx = word_index.get(t, 2)        # get word id or <UNK>
        if idx >= max_words:              # too rare? mark as unknown
            idx = 2
        seq.append(idx)
    return sequence.pad_sequences([seq], maxlen=max_len)  # pad to 200 length

# 🧠 Step 6: Make a prediction (Positive or Negative)
def predict_sentiment(review, threshold=0.5, mdl=None):
    x = encode_review(review)                       # prepare text
    p = float(mdl.predict(x, verbose=0)[0][0])      # get probability
    label = "Positive 😀" if p >= threshold else "Negative 😠"
    return label, p

# 🎬 Step 7: Try it with your trained model
label, prob = predict_sentiment("This movie was absolutely amazing! The story was emotional, the acting was brilliant, and the visuals were stunning", mdl=model); print(label, prob)
label, prob = predict_sentiment("The film was painfully slow, the dialogue was terrible.", mdl=model); print(label, prob)
label, prob = predict_sentiment("It was boring, slow, and poorly written.", mdl=model); print(label, prob)


Positive 😀 0.9924509525299072
Negative 😠 0.07806771993637085
Negative 😠 0.002139448653906584
